In [1]:
#we may need some code in the ../python directory and/or matplotlib styles
import sys
sys.path.append('../python/')

#matplotlib for plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.style.use('../mplstyles/stylelib/standard.mplstyle')

#other computational libraries
import numpy as np

# Summary

The notebook `yieldWidthComputationI.ipynb` mentioned that the exact ionization yield width computation for the GGA3 detector of the EDELWEISS publication [[EDW2004][EDW2004]] was computationally slow. In fact it takes about 1 min to evaluate the yield width for a single energy. Since our fitting will require several thousand function evaluations we sought a way to compute the yield widths of the electron recoil and nuclear recoil bands much more quickly. The *moment expansion* is the method that we use and it was also employed in the original EDELWEISS publication. 

This notebook explains the moment expansion and compares our version to the slightly less accurate version presented in the original EDELWEISS publication. We need to apply some corrections to the nuclear recoil band version of this expansion in order so that the overall deviation from the exact calculation is kept small. 


[EDW2004]: https://doi.org/10.1016/j.nima.2004.04.218 "EDELWEISS 2004 resolution paper"

# The Moment Expansion

The notebook `yieldWidthComputationI.ipynb` showed the definitions of measured recoil energy and measured ionization yield, they are repeated below for convenience:

\begin{equation}
\tilde{E}_r = E_r + \chi \delta H - \Omega \delta I,
\end{equation}

\begin{equation}
Q = \frac{\epsilon N + \delta I}{E_r + \chi \delta H - \Omega \delta I}.
\end{equation}

Here, we have made the following definitions for brevity:

\begin{equation}
\chi \equiv \left(1 + \frac{V}{\epsilon}\right),
\end{equation}

\begin{equation}
\Omega \equiv \left(\frac{V}{\epsilon}\right).
\end{equation}

In what follows--like in the exact computations presented in `yieldWidthComputationI.ipynb`--it is important to note that the random variables $\delta H$, $\delta I$, and $N$ are independent (have covariances of *zero*) and are all plausibly modeled as normally distributed.

In this research we are constructing methods to measure $\sigma_N$ for nuclear recoils given knowledge of $\sigma_I$ and $\sigma_H$, along with experimental measurements of $\sigma_Q$.

As in the exact calculation we can compute $\sigma_Q$ from its definition:

\begin{equation}
\sigma_Q^2 = E[Q^2] - E[Q]^2, 
\end{equation}

where the $E[\cdot]$ is the *expectation* of a quantity given its distribution. 

The previous exact calculations were focused on finding the true PDF of $Q$, i.e. its distribution function at a given measured energy ($f_Q(Q|\tilde{E}_r)$). With that distribution the appropriate integrals are computed to arrive at the variance ($\sigma_Q^2(\tilde{E}_r)$) as defined above.

In the *moment expansion* each expectation value $E[Q]$ and $E[Q^2]$ are computed by doing a multi-dimensional Taylor Expansion of the definition of $Q$ and then taking the expectation values, inserting the appropriate values for quantities like $E[N]$ or $E[(\delta H)^2]$. 

## Computing the Moments

Approximating the following expectations is what we need to do.

\begin{equation}
E[Q] = E\left[\frac{\epsilon N + \delta I}{E_r + \chi \delta H - \Omega \delta I}\right]
\end{equation}

\begin{equation}
E[Q^2] = E\left[\left(\frac{\epsilon N + \delta I}{E_r + \chi \delta H - \Omega \delta I}\right)^2\right]
\end{equation}

The nonlinear functions inside the expectations are replaced with their Taylor Expansions about the means, this is the essence of the moment expansion [[MEWiki][MEWiki]]. 

When the Taylor Expansion is made it leaves the relevant expectation values in the following form:

\begin{equation}
E[Q]^2 = E\left[\left( \bar{Q} + Q_H \delta H + Q_I \delta I + Q_N \left(N-\bar{N}\right) \ldots \right) \right]^2,
\end{equation}

\begin{equation}
E[Q^2] = E\left[\left( \bar{Q} + Q_H \delta H + Q_I \delta I + Q_N \left(N-\bar{N}\right) \ldots \right) \left( \bar{Q} + Q_H \delta H + Q_I \delta I + Q_N \left(N-\bar{N}\right) \ldots \right)\right].
\end{equation}

Here we have used several facts to simplify the expressions inside the expectations slightly. First, the means of the $\delta H$ and $\delta I$ RVs are zero. Next, the notation $Q_I$ or $Q_H$ are shorthand for the first partial derivatives of $Q$ with respect to $\delta I$ and $\delta H$ respectively, evaluated at the mean values for all the RVs. Last, the quantity $\bar{N}$ is simply $E_r\bar{Q}/\epsilon_{\gamma}$ or, in words, the number of electron hole pairs produced by a recoil at a given recoil energy is the average ionization yield evaluated at that energy times the recoil energy and divided by the average electron-equivalent energy to create a single electron-hole pair ($\epsilon_{\gamma}$). 

Because all the basic RVs are normally distributed and independent, there are a number of other simplifications. 

  * All covariances are zero--this greatly reduces the number of terms that contribute to $E[Q^2]$. 
  * All RVs have only even central moments non-zero, and those are related to the second central moment in a well-defined way. 
  * Most terms in $E[Q]^2$ have counterpart terms in $E[Q^2]$ which will perfectly cancel after subtraction. Some examples are terms that involve the leading $\bar{Q}$ of the expansion. 
  * All terms involving derivatives of $N$ will involve the effective nuclear recoil Fano factor and will vanish if we do not include this (like when evaluating the yield width for electron recoils where the "Fano" part of $\sigma_N$ is implicitly included in the resolutions and very small). *__This fact will be important in the extraction of the effective Fano factor in this work__*. 
  
  

[MEWiki]: https://en.wikipedia.org/wiki/Taylor_expansions_for_the_moments_of_functions_of_random_variables "Wikipedia entry for the Moment Expansion"

## Results

Note that the summary of the resolutions in the above is relative to the keVee scale, but we want these, instead, to be quoted as a function of the true recoil energy, $E_r$, for a given ionization yield model. We can use the yield model used in the Edelweiss paper:

\begin{equation}
\bar{Q}(E_r) = aE_r^b,
\end{equation}

with a=0.16 and b=0.18. In terms of the average ionization yield, the conversion of the resolution functions to the $E_r$ scale can be accomplished by transforming the $E_r$ energy into the _electron equivalent_ energy for ionization ($E_{\mathrm{ee}}^I$) or heat ($E_{\mathrm{ee}}^h$). 

\begin{equation}
E_{\mathrm{ee}}^h = \frac{\left(1+ \frac{V}{\epsilon}\bar{Q}\right)E_r}{1+\frac{V}{\epsilon}}
\end{equation}

\begin{equation}
E_{\mathrm{ee}}^I = \bar{Q}E_r
\end{equation}

Note that these two definitions are equivalent when the average ionization yield is one (like for electron-recoils), but differs when the average ionization yield is not one. This underscores the fact that heat and ionization scales cannot be universally (for all recoil types) defined because the average ionization yield does not affect "primary" phonon production and Luke-Neganov phonon production [[Luke88][Luke88]][[Neganov85][Neganov85]] in the same way. 

[Luke88]: https://doi.org/10.1016/0168-9002(90)91510-I "Voltage amplification I"
[Neganov85]: https://link.springer.com/article/10.1134/1.1423744 "Voltage amplification II:"

__Note: [[Neganov85][Neganov85]] is not the correct reference, search for the reference B. Neganov and V. Trofimov, Otkrytiya, Izobret.146, 215(1985), which is cited there; but it's hard to find electronically.__

# Convergence

The moment expansion is a formal tool but it is not known to converge to the correct full expression that we've derived in `yieldWidthComputationI.ipynb`. In fact it may not converge at all but rather be asymptotic. We then want to show that it is a good enough expression for our purposes if we take it to a finite order. 

The first comparison would be the lowest non-trivial order for the expression (used in the EDELWEISS paper [[EDW2004][EDW2004]]) and our higher order approximation for Electron Recoils. 

[EDW2004]: https://doi.org/10.1016/j.nima.2004.04.218 "EDELWEISS 2004 resolution paper"

In [2]:
import prob_dist as pd

F=10
Etest=10
P = pd.QEr_v2_2D_fast(heatRes_GGA3_Er,sigI_GGA3_Er,V,eps,F,Qbar)




#print(P(0.2,10,10))

#check for normalization
from scipy import integrate
#norm = integrate.dblquad(P1, 0, 10, 0, 200,args=(40,))
#print(norm)

#only integrate over important part of distribution around Etr
#I empirically found in analysis/misc/nrFano_Constraint/extract_Edw_Fano_v2.ipynb
#that at Etr=10keV the width should be 3 keV and at 40 keV it should be 10 keV

m = (10-3.0)/(40-10)
icpt = 3-m*10
width = lambda Etr: m*Etr + icpt



lowlim=Etest-3*width(Etest)
uplim=Etest+3*width(Etest)
Pq = lambda Q: integrate.quad(lambda Etr,Q,Er:P(Q,Etr,Er),lowlim,uplim,args=(Q,Etest,))[0]
print(Pq(0.2))
norm = integrate.quad(Pq,-0.2,4)
print(norm)

NameError: name 'heatRes_GGA3_Er' is not defined

In [ ]:
#set up a 1d plot
fig,axes = plt.subplots(1,1,figsize=(9.0,8.0),sharex=True)
ax1 = axes

Pqv = np.vectorize(Pq)

X = np.arange(0,1,0.005)
Y = Pqv(X)


ax1.plot(X,Y,color='orange',linestyle='-',label='{} keV Edelweiss GGA3'.format(Etest))


ax1.axvline(Qbar(Etest), color='r', linestyle='--', lw=2, alpha=0.8,label='predicted center')

ymin = 1e-4
max1 = np.max(Y)
ymax = max1

ax1.set_yscale('linear')
ax1.set_yscale('log')
ax1.set_xlim(0, 1) 
ax1.set_ylim(ymin,ymax)
ax1.set_xlabel(r'ionization yield',**axis_font)
ax1.set_ylabel('PDF',**axis_font)
ax1.grid(True)
#ax1.yaxis.grid(True,which='minor',linestyle='--')
ax1.legend(loc=1,prop={'size':22})

for axis in ['top','bottom','left','right']:
  ax1.spines[axis].set_linewidth(2)

plt.tight_layout()
#plt.savefig('figures/figure.png')
plt.show()

In [ ]:
#let's try to get a look at a contour plot
PQEr = lambda Q,Etr:P(Q,Etr,Etest)
xnr = np.linspace(Etest-3*width(Etest), Etest+3*width(Etest), 300)
ynr = np.linspace(0.0, 0.7, 300)

Xnr, Ynr = np.meshgrid(xnr, ynr)
PQEr_v = np.vectorize(PQEr)
Znr = PQEr_v(Ynr,Xnr)

In [ ]:
#set up a 1d plot
fig,axes = plt.subplots(1,1,figsize=(9.0,8.0),sharex=True)
ax1 = axes

#make a model for the line I think the max probability should lie on
Xhy = np.arange(Etest-3*width(Etest),Etest+3*width(Etest),0.01)
print(Xhy)
fhy = lambda x: Qbar(Etest)*Etest/x
#well, this doesn't seem to predict the curve matching the max in the 2D distribution, but this is
#the naive estimate not accounting for correlations, the question is:
#along which line in the Y,Etr space should the distribution be for a given true recoil energy Etest

#plot quick
#fig = plt.figure()
#ax1 = fig.add_subplot(111)
#ax1.plot(x, ingrndv_alt(x),label='E$_r$ function')
#ax1.contour(X, Y, Z, colors='black',label='E$_r$/N')
ax1.contour(Xnr, Ynr, Znr, 50,cmap='RdGy');
#ax1.plot(Xhy,fhy(Xhy),label='Max proability',color='goldenrod',linestyle='--')


ax1.set_xscale('linear')
ax1.set_yscale('linear')
ax1.set_xlim(Etest-3*width(Etest), Etest+3*width(Etest)) 
ax1.set_ylim(0.0,0.7)
ax1.set_xlabel('recoil energy [keV]',**axis_font)
ax1.set_ylabel('ionization yield',**axis_font)
ax1.grid(True)
ax1.yaxis.grid(True,which='minor',linestyle='--')
#ax1.legend(loc=2,prop={'size':22})

for axis in ['top','bottom','left','right']:
  ax1.spines[axis].set_linewidth(2)

plt.tight_layout()
#plt.savefig('figures/figures.png')
plt.show()

## Adding in the True $E_r$ Distribution

Everything in the preceeding sections made calculations for a fixed true $E_r$. When measurements are executed in a real world device, even the true energy is in some sense a random variable, with a probability distribution dependent on the probability of a neutron (say) to impart a specific recoil energy to a nuclei that it scatters off of. 

Since this random variable is clearly independent of all the random variables studied so far (those have to do with detector fluctuations and electronic and/or analysis noise) we can construct the joint distribution $P(Q,\tilde{E}_r,E_r)$ at once:

\begin{equation}
P(Q,\tilde{E}_r,E_r) = P(Q,\tilde{E}_r|E_r)P(E_r),
\end{equation}

Where $P(E_r)$ is the probability density function of the true recoils. 

A plausible model for $P(E_r)$ is a pure exponential with a decay constant around 1/100 keV$^{-1}$. This can model in a rough sense broadband neutron sources like $^{252}$Cf. In that case the probability distribution of the true $E_r$ values is:

\begin{equation}
P(E_r) = \frac{1}{\alpha}e^{-\alpha E_r},
\end{equation}

where $\alpha$ is the decay constant and we take to be equal to about 1/100 keV$^{-1}$. Using this model we can immediately find the two-dimensional experimental Q,$\tilde{E}_r$ distribution for such a nuclear recoil source, it is:

\begin{equation}
P(Q,\tilde{E}_r) = \int_0^{\infty} dE_r P(Q,\tilde{E}_r|E_r)P(E_r).
\end{equation}

With the functions constructed in the previous section we have all the ingredients to compute this distribution. Simply take the two-dimensional function returned from `QEr_v2_2D_fast` in the library `../prob_dist.py` and that is a three-argument function which is equal to $P(Q,\tilde{E}_r|E_r)$ with arguments in the order Q,$\tilde{E}_r$,$E_r$. Using that and our above definition of $P(E_r)$ we can get the desired distribution. 

**NOTE: The function $P(Q,\tilde{E}_r|E_r)$ is difficult to numerically integrate over all $E_r$ for a fixed $\tilde{E}_r$ I think because it is sharply peaked near $E_r$=$\tilde{E}_r$. Therefore, as above, I typically empirically find limits for the $E_r$ based on the value of $\tilde{E}_r$ which approximate the integral quite well but doesn't turn the integrand into a function that is defined one a wide range but only non-zero on a set of very small measure compared to the domain.** 

There is another function in the library `../prob_dist.py`, `expband_2D` that can return the two-dimensional function given the three-dimensional proability distribution $P(Q,\tilde{E}_r,E_R)$, and $\alpha$ as defined above. 

In [ ]:
#Get two-dimensional band function
nr_band = pd.expband_2D(P,(1/100))

xnr = np.linspace(1, 100, 200)
ynr = np.linspace(0.0, 0.7, 200)

Xnr, Ynr = np.meshgrid(xnr, ynr)
nr_band_v = np.vectorize(nr_band)
Znr = nr_band_v(Ynr,Xnr)

In [ ]:
#set up a 1d plot
fig,axes = plt.subplots(1,1,figsize=(9.0,8.0),sharex=True)
ax1 = axes


#plot quick
#fig = plt.figure()
#ax1 = fig.add_subplot(111)
#ax1.plot(x, ingrndv_alt(x),label='E$_r$ function')
#ax1.contour(X, Y, Z, colors='black',label='E$_r$/N')
ax1.contour(Xnr, Ynr, Znr, 50,cmap='RdGy');
#ax1.plot(Xhy,fhy(Xhy),label='Max proability',color='goldenrod',linestyle='--')


ax1.set_xscale('linear')
ax1.set_yscale('linear')
ax1.set_xlim(0, 100) 
ax1.set_ylim(0.0,0.7)
ax1.set_xlabel('recoil energy [keV]',**axis_font)
ax1.set_ylabel('ionization yield',**axis_font)
ax1.grid(True)
ax1.yaxis.grid(True,which='minor',linestyle='--')
#ax1.legend(loc=2,prop={'size':22})

for axis in ['top','bottom','left','right']:
  ax1.spines[axis].set_linewidth(2)

plt.tight_layout()
#plt.savefig('figures/figures.png')
plt.show()